In [1]:
# packages for dealing with Sidra API
import requests
import json

# packages for treating the data
import pandas as pd
import pandana as pdna
from pandana.loaders import osm
import numpy as np
import unidecode

# packages for dealing with geographical data
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import osmnx as ox
from osmnx.distance import get_nearest_node


## Building the dataset

This project aims to measure the pedestrian acessibility in Natal, Brazil. With that in mind, we use data from [Sidra API](http://api.sidra.ibge.gov.br/) regarding the neighborhoods, average income and population size. Further, we calculate the smallest distance from a central point of the neighborhood to an amenity. All this informations togheter will become our dataset.

In [2]:
# Recover income by neighborhood in Natal (2010)
# from Sidra's Table 3170 (http://api.sidra.ibge.gov.br/desctabapi.aspx?c=3170)
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

endnode = "http://api.sidra.ibge.gov.br/values/t/3170/p/2010/v/842/N102/in%20n6%202408102"
response = requests.get(endnode,headers=headers)
raw_data_income = response.json()

In [3]:
# Recover the number of inhabitants
endnode_people = "http://api.sidra.ibge.gov.br/values/t/3170/p/2010/v/841/N102/in%20n6%202408102"
response = requests.get(endnode_people,headers=headers)
raw_data_people = response.json()

The variables 842 and 841 give us, respectivelly, the information about income and population of each neighborhood. Now we take a look at the responses to find out the meaning of each key-value pair.

In [4]:
raw_data_income[0]

{'NC': 'Nível Territorial (Código)',
 'NN': 'Nível Territorial',
 'MC': 'Unidade de Medida (Código)',
 'MN': 'Unidade de Medida',
 'V': 'Valor',
 'D1C': 'Ano (Código)',
 'D1N': 'Ano',
 'D2C': 'Variável (Código)',
 'D2N': 'Variável',
 'D3C': 'Bairro (Código)',
 'D3N': 'Bairro',
 'D4C': 'Situação do domicílio (Código)',
 'D4N': 'Situação do domicílio',
 'D5C': 'Sexo (Código)',
 'D5N': 'Sexo',
 'D6C': 'Grupo de idade (Código)',
 'D6N': 'Grupo de idade'}

In [5]:
raw_data_people[0]

{'NC': 'Nível Territorial (Código)',
 'NN': 'Nível Territorial',
 'MC': 'Unidade de Medida (Código)',
 'MN': 'Unidade de Medida',
 'V': 'Valor',
 'D1C': 'Ano (Código)',
 'D1N': 'Ano',
 'D2C': 'Variável (Código)',
 'D2N': 'Variável',
 'D3C': 'Bairro (Código)',
 'D3N': 'Bairro',
 'D4C': 'Situação do domicílio (Código)',
 'D4N': 'Situação do domicílio',
 'D5C': 'Sexo (Código)',
 'D5N': 'Sexo',
 'D6C': 'Grupo de idade (Código)',
 'D6N': 'Grupo de idade'}

The keys for neighborhood code, neighborhood name, and value for each variable (income and population), are, in this order, `D3C`, `D3N` and `V`. We use these to build the first dataframes. Only the dataset for income will have the name, because the two datasets will be merged using the neighboorhood code.

In [6]:
# create a dataset for each json
income_df = pd.DataFrame(raw_data_income, columns=['D3C', 'D3N', 'V'])
income_df.drop(labels=0, inplace=True)

people_df = pd.DataFrame(raw_data_people, columns=['D3C', 'V'])
people_df.drop(labels=0, inplace=True)

In [7]:
# quick checkout on the results
print(income_df.head())
print(people_df.head())

          D3C                         D3N        V
1  2408102001    Santos Reis - Natal - RN   984.31
2  2408102002  Praia do Meio - Natal - RN  1658.16
3  2408102003          Rocas - Natal - RN   969.39
4  2408102004        Ribeira - Natal - RN  2825.82
5  2408102005     Petrópolis - Natal - RN  4736.63
          D3C     V
1  2408102001  2989
2  2408102002  2810
3  2408102003  5806
4  2408102004  1453
5  2408102005  3288


In [8]:
neighborhood_df = income_df.merge(people_df, on='D3C')
neighborhood_df.head()

,D3C,D3N,V_x,V_y
0,2408102001,Santos Reis - Natal - RN,984.31,2989
1,2408102002,Praia do Meio - Natal - RN,1658.16,2810
2,2408102003,Rocas - Natal - RN,969.39,5806
3,2408102004,Ribeira - Natal - RN,2825.82,1453
4,2408102005,Petrópolis - Natal - RN,4736.63,3288


Finally, the columns will be renamed. There is a need to remove any accents from the names of the neighborhoods. This is necessary to enable the merge with the geojson data, that only has names, not codes for identifying the regions. Also, the redundant "- Natal - RN" must be removed too.

In [9]:
new_columns = {'V_x':'income', 'D3C':'id', 'D3N':'name', 'V_y':'inhabitants'}
neighborhood_df.rename(columns=new_columns, inplace=True)

# normalize the names
neighborhood_df['name'] = neighborhood_df.name.str.split(pat=' - ', n=1, expand=True)[0]
neighborhood_df['name_norm'] = neighborhood_df.name.apply(lambda x:unidecode.unidecode(x))

In [10]:
neighborhood_df.head()

,id,name,income,inhabitants,name_norm
0,2408102001,Santos Reis,984.31,2989,Santos Reis
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio
2,2408102003,Rocas,969.39,5806,Rocas
3,2408102004,Ribeira,2825.82,1453,Ribeira
4,2408102005,Petrópolis,4736.63,3288,Petropolis


Last but not least, we will add the administrative regions or zones, as they are known, to which the districts belong to the Ordinary Law nº 03878/89 of the municipality of Natal. This information was previously inserted into a JSON file

In [11]:
with open('../data/interim/zones.json') as fp:
    zones = json.load(fp)
neighborhood_df['zone'] = neighborhood_df['name'].map(zones) 

In [12]:
neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East
2,2408102003,Rocas,969.39,5806,Rocas,East
3,2408102004,Ribeira,2825.82,1453,Ribeira,East
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East


### Retrieving the centroid of each neighborhood
The GeoJSON file provides a polygon that describes Natal geographically. If we get its central point, which means its centroid, we can measure the distance between the center of the area and a point of interest as school, bank, police station, etc. [geopandas](http://geopandas.org/) module is responsible for dealing with this data.

In [13]:
# load the GeoJSON data and use 'UTF-8'encoding
geojson_natal_file = requests.get('https://github.com/nymarya/data-science-one/blob/master/Lesson%2314/natal.geojson?raw=true')
geo_json_natal = geojson_natal_file.json()

In [14]:
# save it in a local file
with open('../data/external/natal.geojson', 'w') as file:  
    json.dump(geo_json_natal, file)
    
geojson_df = gpd.read_file("../data/external/natal.geojson")

Let's make sure that the data is all there

In [15]:
geojson_df.head()

,id,@id,admin_level,boundary,is_in,name,place,type,alt_name,wikidata,geometry
0,relation/388146,relation/388146,10,administrative,Natal,Pitimbu,suburb,boundary,None,None,"POLYGON ((-35.24373 -5.84203, -35.24379 -5.842..."
1,relation/388147,relation/388147,10,administrative,Natal,Planalto,suburb,boundary,None,None,"POLYGON ((-35.25382 -5.86629, -35.25361 -5.866..."
2,relation/397022,relation/397022,10,administrative,Natal,Ponta Negra,suburb,boundary,None,None,"POLYGON ((-35.18902 -5.89079, -35.18895 -5.890..."
3,relation/1230018,relation/1230018,10,administrative,Natal,Neópolis,suburb,boundary,None,None,"POLYGON ((-35.20031 -5.87341, -35.19996 -5.873..."
4,relation/1230020,relation/1230020,10,administrative,Natal,Capim Macio,suburb,boundary,None,None,"POLYGON ((-35.18946 -5.87242, -35.18907 -5.871..."


In [16]:
geojson_df.name.unique()

array(['Pitimbu', 'Planalto', 'Ponta Negra', 'Neópolis', 'Capim Macio',
       'Lagoa Azul', 'Pajuçara', 'Lagoa Seca', 'Barro Vermelho',
       'Candelária', 'Praia do Meio', 'Rocas', 'Santos Reis', 'Redinha',
       'Salinas', 'Igapó', 'Nossa Senhora da Apresentação', 'Potengi',
       'Ribeira', 'Cidade Alta', 'Alecrim', 'Nordeste', 'Quintas',
       'Bom Pastor', 'Dix-Sept Rosado', 'Nossa Senhora de Nazaré',
       'Lagoa Nova', 'Mãe Luiza', 'Nova Descoberta', 'Tirol',
       'Petrópolis', 'Areia Preta', 'Cidade Nova', 'Cidade da Esperança',
       'Felipe Camarão', 'Guarapes'], dtype=object)

In [17]:
n_neighborhoods1 = len(neighborhood_df.name.unique())
n_neighborhoods2 = len(geojson_df.name.unique())
assert n_neighborhoods1 == n_neighborhoods2

Knowing that the polygon is a complex structure/data type, merging the datasets using pandas is also complex. So in this step, we merge the datasets and then extract the x and y.

In [18]:
# fix typo
typo_df = geojson_df.query('name == "Felipe Camarão"')
geojson_df.loc[typo_df.index.values, 'name'] = 'Filipe Camarão'

# save each coordinate
geojson_df['name_norm'] = geojson_df.name.apply(lambda x:unidecode.unidecode(x))
neighborhood_df = neighborhood_df.merge(geojson_df[['name_norm', 'geometry']], on='name_norm')


neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone,geometry
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East,"POLYGON ((-35.20819 -5.76785, -35.20420 -5.768..."
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East,"POLYGON ((-35.19504 -5.77323, -35.19513 -5.773..."
2,2408102003,Rocas,969.39,5806,Rocas,East,"POLYGON ((-35.19664 -5.77523, -35.19653 -5.774..."
3,2408102004,Ribeira,2825.82,1453,Ribeira,East,"POLYGON ((-35.20412 -5.76860, -35.20420 -5.768..."
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East,"POLYGON ((-35.19536 -5.78434, -35.19532 -5.784..."


In [19]:
# create columns in the main df for the centroid
neighborhood_df['x'] = neighborhood_df.geometry.apply(lambda x: Polygon(x).centroid.x) # lon
neighborhood_df['y'] = neighborhood_df.geometry.apply(lambda x: Polygon(x).centroid.y) # lat 

neighborhood_df.head()

,id,name,income,inhabitants,name_norm,zone,geometry,x,y
0,2408102001,Santos Reis,984.31,2989,Santos Reis,East,"POLYGON ((-35.20819 -5.76785, -35.20420 -5.768...",-35.199888,-5.762731
1,2408102002,Praia do Meio,1658.16,2810,Praia do Meio,East,"POLYGON ((-35.19504 -5.77323, -35.19513 -5.773...",-35.195083,-5.778860
2,2408102003,Rocas,969.39,5806,Rocas,East,"POLYGON ((-35.19664 -5.77523, -35.19653 -5.774...",-35.200216,-5.774142
3,2408102004,Ribeira,2825.82,1453,Ribeira,East,"POLYGON ((-35.20412 -5.76860, -35.20420 -5.768...",-35.205768,-5.776236
4,2408102005,Petrópolis,4736.63,3288,Petropolis,East,"POLYGON ((-35.19536 -5.78434, -35.19532 -5.784...",-35.198393,-5.784591


### Calculating the distance
After a search for the bbox coordinates in [https://boundingbox.klokantech.com](https://boundingbox.klokantech.com), we create a [Pandana](https://udst.github.io/pandana/network.html) network for Natal. Pandana provides efficent ways to calculate distances.

In [20]:
bbox=[-5.90021100,-35.29126300,-5.70272200,-35.1531030]

In [ ]:
net = osm.pdna_network_from_bbox(*bbox)

net.precompute(5000)
net.edges_df.head(2)

Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](-5.90021100,-35.29126300,-5.70272200,-35.15310300);>;);out;'}"
Downloaded 12,474.1KB from www.overpass-api.de in 4.27 seconds
Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 4.54 seconds
Returning OSM data with 84,321 nodes and 21,376 ways...


A quick test: retrieving the centroid of a neighborhood, and using `get_node_ids(x, y)` to get the node in the network closer to the point. That way, we can check the osmid and see if the node is really in the center of the area.

In [ ]:
nordeste = neighborhood_df.query("name == 'Nordeste'")
x = nordeste.x.tolist()[0]
y = nordeste.y.tolist()[0]
neighborhood_node_id = net.get_node_ids([x], [y])
neighborhood_node_id

The result is available [here](https://overpass-turbo.eu/s/190G).

#### Testing pandana `nearest_pois`

Let's make a quick test: search amenities for *one* neighborhood and see if we can calculate the distances. Code mostly from [Pandana demo](https://github.com/UDST/pandana/blob/dev/examples/Pandana-demo.ipynb)

In [ ]:
schools = osm.node_query(*bbox, tags='"amenity"="school"')
schools.head(2)

In [ ]:
# set_pois receives longitude (x) and then latitude (y)
net.set_pois("schools", 10000, 10, schools.lon, schools.lat)


In [ ]:
# get the 10 nearest schools for each node
result = net.nearest_pois(10000, "schools", num_pois=10)
result

We can find the smallest distance for the point retrieved earlier, the centroid of Nordeste, and get the distances for the 10 nearest schools around it.

In [ ]:
neighborhood_node_id = net.get_node_ids([x], [y])
neighborhood_node_id

In [ ]:
result.loc[neighborhood_node_id[0],:]

#### Calcutate the distance

Now, we can repeat all these steps for all neighborhoods and save them

In [ ]:
amenities = ['hospital', 'restaurant', 'bank','school', 'police', 'pharmacy']
# create string for query that must be in form: "amenity"~"hospital|bank..."
amenities_tag = '|'.join(amenities)
print(amenities_tag)

# create new columns
cols = ['nearest_' + amenity for amenity in amenities]
neighborhood_df = neighborhood_df.reindex(columns=neighborhood_df.columns.tolist() + cols)

In [ ]:
all_pois = osm.node_query(*bbox, tags=f'"amenity"~"{amenities_tag}"')
for amenity in amenities:
    pois = all_pois.query(f"amenity == '{amenity}'")
    # set pois for amenity
    net.set_pois(amenity, 10000, 10, pois.lon, pois.lat)
    
    # get 10 nearest pois
    result = net.nearest_pois(10000, amenity, num_pois=10)
    
    
    for name in neighborhood_df.name.values:
        # get centroid
        neighborhood = neighborhood_df.query(f"name == '{name}'")
        x = neighborhood.x.tolist()[0]
        y = neighborhood.y.tolist()[0]
        
        # get node for centroid
        neighborhood_node_id = net.get_node_ids([x], [y])
        
        # get distance between closer amenity and centroid 
        distance = result.loc[neighborhood_node_id[0], 1]
        # update dataframe
        neighborhood_df.loc[neighborhood.index, f'nearest_{amenity}'] = distance

In [ ]:
neighborhood_df.head()

Finally, save dataset to use it in the the next step: creating maps.

In [ ]:
neighborhood_df.to_csv('../data/processed/neighbourhoods.csv')

### Extra: create a visualization
Our goal is to compare the districts, which is not possible using Pandana. However, we can get a initial idea for the whole city. We will count how many schools there are 5km around each node.

In [ ]:
# set variables
school_nodes = net.get_node_ids(schools.lon, schools.lat)
net.set(school_nodes, name = 'schools')
accessibility = net.aggregate(distance = 10000,
                              type = 'count',
                              name = 'schools')

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(8,10))
ax.set_facecolor('black')
plt.title('Natal: Schools within 10km')
plt.scatter(net.nodes_df.x, net.nodes_df.y, 
            c=accessibility, s=1, cmap='YlOrRd')
cb = plt.colorbar()
plt.savefig('../reports/figures/distances_cmap1.png')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,18))
ax.set_facecolor('black')
plt.scatter(net.nodes_df.x, net.nodes_df.y, 
            c=accessibility, s=1, cmap='YlOrRd', facecolor='black')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.savefig('../reports/figures/distances_cmap.png')
plt.show()